In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate qwen-vl-utils bitsandbytes

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-sheta28u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-sheta28u
  Resolved https://github.com/huggingface/transformers to commit 3927ffed31e3c0d2929bf98bd05b7c61fcc48b62
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.0/502.0 kB 9.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [4]:
from transformers import pipeline
from datasets import Dataset

# Carica il modello BART per il riassunto
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_text(x_t: str, max_length: int = 150, min_length: int = 50) -> str:
    """
    Riassume il testo utilizzando il modello BART.
    """
    # Usa il modello per ottenere un riassunto
    summary = summarizer(x_t, max_length=max_length, min_length=min_length, do_sample=False)
    return summary[0]['summary_text']

# Funzione per applicare il riassunto a tutto il dataset
def add_summary_to_dataset(example):
    # Applica il riassunto alla descrizione x_t
    summarized_x_t = summarize_text(example['x_t'])
    return {'summarized_x_t': summarized_x_t}



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [4]:
# Test
# print("Description before summary: ", ds_loaded[0]['x_t'])
# summary = summarize_text(ds_loaded[0]['x_t'])
# print("Summary obtained: ", summary)

Your max_length is set to 150, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)


Description before summary:  The scene depicts a woman sitting on a couch in a dimly lit living room, surrounded by soft pastel colors. The overall brightness is consistent, and the camera is steady. Some fine details, such as the woman's hair, are slightly sharpened, but they blend well into the background. There is a subtle but noticeable difference in brightness between the woman and the background, hinting at possible post-processing. However, the lighting is consistent, and no significant glares or shadows are present. The scene appears to be captured during daytime, as some outdoor rays can be seen peaking
Summary obtained:  The scene depicts a woman sitting on a couch in a dimly lit living room. Some fine details, such as the woman's hair, are slightly sharpened, but they blend well into the background. There is a subtle but noticeable difference in brightness between the woman and the background, hinting at possible post-processing.


In [5]:
import shutil

src = "/kaggle/input/new-correct-dataset-2/kaggle/working/new_correct_dataset_2"
dst = "/kaggle/working/test_set_without_xt_2"

shutil.copytree(src, dst, dirs_exist_ok=True)
print(f"Copiato dataset da {src} a {dst}")


Copiato dataset da /kaggle/input/new-correct-dataset-2/kaggle/working/new_correct_dataset_2 a /kaggle/working/test_set_without_xt_2


In [6]:
ds_loaded = Dataset.load_from_disk(dst)

In [10]:
print(ds_loaded['image'][0])

new_data/real/auto_factory_3.jpg


In [ ]:
# Applica la funzione di riassunto al dataset
ds_with_summary = ds_loaded.map(add_summary_to_dataset, batched=False)

# Mostra alcuni esempi per verificare il riassunto
print(ds_with_summary['summarized_x_t'][:5])
ds_with_summary.save_to_disk("/kaggle/working/new_dataset_summarized")